In [1]:
import pandas as pd
import numpy as np

Will have to make some code here to handle the original file format if use an updated version  
Think I manually dropped the first week 1 - 11 games in excel  
Also deleted some columns 
Manually gave the home/away column name - OG data has no column name for it

### Create home and away team columns from data
Data comes in with winner and loser teams vs home and away  
Have to use the home/away column

In [2]:

def get_home_and_away_columns(sched_df):
    sched_df['home'] = np.where(
        sched_df['home/away'] == '@', 
        sched_df['Loser/tie'], 
        sched_df['Winner/tie']
    )
    sched_df['away'] = np.where(
        sched_df['home/away'] == '@', 
        sched_df['Winner/tie'], 
        sched_df['Loser/tie']
    )

    return sched_df

### Check if any home and away teams are the same value
If ```True in Same``` returns True than have an issue 

### Remove City from Team Name
Also convert to all lowercase

In [3]:

def get_short_team_name(full_team_name):
    short_name = full_team_name.rsplit(None, 1)[-1]
    lc_short_name = short_name.lower()
    return lc_short_name


### Get Home and Away Score 
Stored in original file as Points Winner (PtsW) and Points Loser (PtsL)

In [4]:
# sched_df['home_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsL'], sched_df['PtsW'])
# sched_df['away_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsW'], sched_df['PtsL'])


#### Drop unused columns

In [5]:
# sched_df = sched_df.drop(columns=['Winner/tie','home/away','Loser/tie','PtsW','PtsL'])


### [datetime docs]('https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html')

## Create Game Time Column based on date and time columns

In [6]:
# sched_df['GameTime'] = pd.to_datetime(sched_df['Date'] + 'T' + sched_df['Time'] + '-0500', utc=True )
# print(sched_df.dtypes)
# sched_df.head()

### Create Start and End Times based off Game Time
Start time is 4 days 1 hour before start of game  
End time is 1 hour before start of game

In [7]:
# sched_df['SearchStartTime'] = sched_df['GameTime'] - pd.to_timedelta('4 days 01:00:00')
# sched_df['SearchEndTime'] = sched_df['GameTime'] - pd.to_timedelta('01:00:00')
# sched_df.head()

### Save to CSV

In [8]:

# sched_df.to_csv('./2022_schedule_cleaned.csv', index=False)

In [9]:


def get_weekly_df(filepath, week):
    # import schedule
    sched_df = pd.read_csv(filepath)

    # drop columns
    sched_df = sched_df.drop(['YdsW','TOW','YdsL','TOL', 'Unnamed: 7'], axis=1)

    #rename home/away column    
    sched_df = sched_df.rename(columns={"Unnamed: 5": "home/away"})

    sched_df = get_home_and_away_columns(sched_df)

    # verify no home and away teams the same
    same = np.where(sched_df['home'] == sched_df['away'], True, False)
    if (True in same):
        print('error, same team name in home and away')
        return

    # remove city from team names
    sched_df['home'] = sched_df['home'].apply(lambda x: get_short_team_name(x))
    sched_df['away'] = sched_df['away'].apply(lambda x: get_short_team_name(x))
    
    # create home and away score columns frm ptsl& ptsw
    sched_df['home_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsL'], sched_df['PtsW'])
    sched_df['away_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsW'], sched_df['PtsL'])

    # drop remaining unused columns
    sched_df = sched_df.drop(columns=['Winner/tie','home/away','Loser/tie','PtsW','PtsL'])
    
    # create game time column from date and time col in UTC format
    sched_df['GameTime'] = pd.to_datetime(sched_df['Date'] + 'T' + sched_df['Time'] + '-0500', utc=True )

    # create search start and end time columns
    sched_df['SearchStartTime'] = sched_df['GameTime'] - pd.to_timedelta('4 days 01:00:00')
    sched_df['SearchEndTime'] = sched_df['GameTime'] - pd.to_timedelta('01:00:00')

    # save as csv
    save_filepath = f'./schedules/schedule_wk_{week}_cleaned.csv'

    sched_df.to_csv(save_filepath, index=False)

    return sched_df
    

### Instructions

May need to rerun code to get imports and whatnot

Go to [pro-football.com](https://www.pro-football-reference.com/years/2022/games.htm)    
Export to CSV, highlight the weeks required and copy and paste into sheets file on new sheet  
[sheets-doc](https://docs.google.com/spreadsheets/d/1UHoydeYY3Yfw34PlSWOwtFeh4uwWb9TkUBCs-hY4yzQ/edit#gid=1220878348)  
Download as csv and save into schedules folder in this repo with filename ```schedule_wk_##.csv```
Set pathname to ```schedule_wk_##.csv```  
Set week to correct week  

Run Code

In [11]:
pathname = './schedules/schedule_wk_16.csv'
week = '16'

wk16 = get_weekly_df(pathname, '16')
wk16



     


,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,16,Thu,2022-12-22,8:15 PM,jets,jaguars,3.0,19.0,2022-12-23 01:15:00+00:00,2022-12-19 00:15:00+00:00,2022-12-23 00:15:00+00:00
1,16,Sat,2022-12-24,1:00 PM,bears,bills,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
2,16,Sat,2022-12-24,1:00 PM,ravens,falcons,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
3,16,Sat,2022-12-24,1:00 PM,chiefs,seahawks,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
4,16,Sat,2022-12-24,1:00 PM,vikings,giants,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
5,16,Sat,2022-12-24,1:00 PM,browns,saints,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
6,16,Sat,2022-12-24,1:00 PM,patriots,bengals,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
7,16,Sat,2022-12-24,1:00 PM,titans,texans,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
8,16,Sat,2022-12-24,1:00 PM,panthers,lions,NaN,NaN,2022-12-24 18:00:00+00:00,2022-12-20 17:00:00+00:00,2022-12-24 17:00:00+00:00
9,16,Sat,2022-12-24,4:05 PM,49ers,commanders,NaN,NaN,2022-12-24 21:05:00+00:00,2022-12-20 20:05:00+00:00,2022-12-24 20:05:00+00:00
